###  Importing all required libraries
---





In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import nltk
from nltk.tokenize import word_tokenize, sent_tokenize
nltk.download('punkt')
import numpy as np
from nltk.corpus import stopwords
nltk.download('stopwords')
import string
import re

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [15]:
!mkdir Text_Analysis_Project


mkdir: cannot create directory ‘content/drive/Text_Analysis_Project’: No such file or directory


In [19]:
cd /content

/content


In [20]:
! ls -1
! zip content.zip *.txt

100.txt
101.txt
102.txt
103.txt
104.txt
105.txt
106.txt
107.txt
108.txt
109.txt
10.txt
110.txt
111.txt
112.txt
113.txt
114.txt
115.txt
116.txt
117.txt
118.txt
119.txt
11.txt
120.txt
121.txt
122.txt
123.txt
124.txt
125.txt
126.txt
127.txt
128.txt
129.txt
12.txt
130.txt
131.txt
132.txt
133.txt
134.txt
135.txt
136.txt
137.txt
138.txt
139.txt
13.txt
140.txt
141.txt
142.txt
143.txt
144.txt
145.txt
146.txt
147.txt
148.txt
149.txt
14.txt
150.txt
15.txt
16.txt
17.txt
18.txt
19.txt
1.txt
20.txt
21.txt
22.txt
23.txt
24.txt
25.txt
26.txt
27.txt
28.txt
29.txt
2.txt
30.txt
31.txt
32.txt
33.txt
34.txt
35.txt
36.txt
37.txt
38.txt
39.txt
3.txt
40.txt
41.txt
42.txt
43.txt
44.txt
45.txt
46.txt
47.txt
48.txt
49.txt
4.txt
50.txt
51.txt
52.txt
53.txt
54.txt
55.txt
56.txt
57.txt
58.txt
59.txt
5.txt
60.txt
61.txt
62.txt
63.txt
64.txt
65.txt
66.txt
67.txt
68.txt
69.txt
6.txt
70.txt
71.txt
72.txt
73.txt
74.txt
75.txt
76.txt
77.txt
78.txt
79.txt
7.txt
80.txt
81.txt
82.txt
83.txt
84.txt
85.txt
86.txt
87.txt
88.t

**Taking all the required websites and generating the files of content from each website**

In [2]:
websites = pd.read_csv('/content/drive/MyDrive/Data/Websites.csv')
output = pd.read_csv('/content/drive/MyDrive/Data/Output.csv')

urls = websites['URL']  # all websites
content_list_of_all_websites = []  # list of content of all websites
personal_pronoun_count = []

headers = {
    "User-Agent": "Mozilla/5.0 (X11; Linux x86_64; rv:60.0) Gecko/20100101 Firefox/60.0"
}

def getContent(urls):    # will create a list of content from all the websites
    for url in urls :
        html_text = requests.get(url, headers=headers)
        soup = BeautifulSoup(html_text.content, 'html.parser')
        contents = soup.find_all('div', class_ = 'td-post-content')
        title = soup.find('header', class_ = 'td-post-title')
        title = title.find('h1').get_text()
        paragraph_clean = title
        personal_pronouns = ['I','we','my','ours','us']
        for data in contents:

            para = data.text
            for pronoun in personal_pronouns:
                pp_count = len(re.findall(pronoun, para))
            personal_pronoun_count.append(pp_count)
            paragraph_clean += para.replace('\xa0','') # removes unwanted text with ''

        try:
             paragraph_clean = paragraph_clean.replace(soup.find('pre').text, '')
        except :
            pass
        content_list_of_all_websites.append(paragraph_clean)

def createFiles():
    for i in range(len(content_list_of_all_websites)):
        file = open(str(i + 1) + '.txt', 'w')
        file.write(content_list_of_all_websites[i])
        file.close()



getContent(urls)  # initilaised the above function

createFiles()     # # creating text files named as respective url_id and saving the content in them









In [9]:
createFiles()

# creating stop words from given folder in drive and removing the stopwords from content








In [12]:

allstopwords = []

def stopWords():
    files = ['/content/drive/MyDrive/Data/StopWords/StopWords_Auditor.txt','/content/drive/MyDrive/Data/StopWords/StopWords_Currencies.txt','/content/drive/MyDrive/Data/StopWords/StopWords_DatesandNumbers.txt','/content/drive/MyDrive/Data/StopWords/StopWords_Generic.txt','/content/drive/MyDrive/Data/StopWords/StopWords_GenericLong.txt','/content/drive/MyDrive/Data/StopWords/StopWords_Geographic.txt','/content/drive/MyDrive/Data/StopWords/StopWords_Names.txt']
    stopwords = []
    for file in files:
        with open(file, 'rb') as f:
            a = f.read()
            stopwords.append(a.decode('latin-1').lower())
    for i in range(len(stopwords)):
        allstopwords.extend(stopwords[i].split())


stopWords()
allstopwords = set(allstopwords)

# tokenization of the content using the stop words created above and creating the list of remaining words(after removing stop words)

Content_withoutStopWords = []  # this contains the list of all the contents after removing the stop words
tokenized_content = []  # list of all the tokenized content

def remove_Stop_Words():
    for content in content_list_of_all_websites:
        tokenizedContent = word_tokenize(content.lower())
        tokenized_content.append(tokenizedContent)
        content_without_stop_words_current = []
        for word in tokenizedContent:
            if word not in allstopwords:
                content_without_stop_words_current.append(word)
        Content_withoutStopWords.append(content_without_stop_words_current)

remove_Stop_Words()

#Total Words after cleaning in each content

total_words_list_after_cleaning = []

for word_list in Content_withoutStopWords:
    total_words_list_after_cleaning.append(len(word_list))
total_words_list_after_cleaning = np.array(total_words_list_after_cleaning)

# print((Content_withoutStopWords))
# print(total_words_list_after_cleaning)




**Creating a dictionary of Positive and Negative words**

Calculating the following scores:


1.   Positive and Negative Score
2.   Polarity Score
3.   Subjectivity Score



In [14]:

positive_words = []
negative_words = []

def pos_neg_words():
    files = ['/content/drive/MyDrive/Data/MasterDictionary/negative-words.txt','/content/drive/MyDrive/Data/MasterDictionary/positive-words.txt']

    for file in files:
        if file == '/content/drive/MyDrive/Data/MasterDictionary/negative-words.txt':
            with open(file, 'rb') as f:
                    a = f.read()
                    negative_words.extend((a.decode('latin-1').lower()).split())
        else:
            with open(file, 'rb') as f:
                    a = f.read()
                    positive_words.extend((a.decode('latin-1').lower()).split())

pos_neg_words()
positive_words = set(positive_words)
negative_words = set(negative_words)

# print(positive_words)
# print(negative_words)

# function to create dictionary of positive and negative words from the content collected from the websites

positive_words_list_from_content = []
negative_words_list_from_content = []

# function to create dictionary of positive and negative words from the content collected from the websites

positive_words_list_from_content = []
negative_words_list_from_content = []

def collect_Words_from_content():
    for words_of_each_content in Content_withoutStopWords:
        pos_words = []
        neg_words = []
        for word in words_of_each_content:
            if word in positive_words:
                pos_words.append(word)
            elif word in negative_words:
                neg_words.append(word)
        positive_words_list_from_content.append(set(pos_words))
        negative_words_list_from_content.append(set(neg_words))

collect_Words_from_content()
# print(positive_words_list_from_content)
# print(negative_words_list_from_content)

## Extracting Derived variables

# Positive And Negative score

PositiveScore = []
NegativeScore = []
def calculate_score():
    for p in positive_words_list_from_content:
        PositiveScore.append(len(p))
    for n in negative_words_list_from_content:
        NegativeScore.append(len(n))

calculate_score()
PositiveScore = np.array(PositiveScore)
NegativeScore = np.array(NegativeScore)
# print(PositiveScore)
# print(NegativeScore)

## Polarity score
Polarity_score = np.divide(np.subtract(PositiveScore,NegativeScore),(np.add(PositiveScore,NegativeScore)+0.000001))

# print(Polarity_score)

# Subjectivity Score

Subjectivity_score = np.divide(np.add(PositiveScore,NegativeScore),(total_words_list_after_cleaning+0.000001))

# print(Subjectivity_score)
# print(total_words_list_after_cleaning)

**Analysis of Readability (using nltk model stopwords) using the Gunning Fox index formula**

Calculating the following in each content:


1.   Total number words 
2.   Average Sentence Length
3.   Average Word Length
2.   Percentage Complex Words(>2 syllables)
1.   Syllables Per Word
2.   Personal Pronoun Count(calculated in 2 tab of this colab)





In [15]:
 
PUNCT_TO_REMOVE = list(string.punctuation)
stpwrds = list(stopwords.words('english'))
PUNCT_TO_REMOVE.extend(stpwrds)
stop_words_from_nlkt = set(PUNCT_TO_REMOVE)

# print(PUNCT_TO_REMOVE)

Content_withoutStopWords_nltk = []  # this contains the list of all the contents after removing the stop words
total_character_count_after_cleaning = []
def remove_Stop_Words_nltk():
    for words in tokenized_content:
        content_without_stop_words_current = []
        count = 0
        for word in words:
            if word not in PUNCT_TO_REMOVE:
                content_without_stop_words_current.append(word)
                count += len(word)
        Content_withoutStopWords_nltk.append(content_without_stop_words_current)
        total_character_count_after_cleaning.append(count)
remove_Stop_Words_nltk()

# print(Content_withoutStopWords_nltk)

#Total Words after cleaning in each content using stopwords from nltk library

total_words_list_after_cleaning_nltk = []

for word_list in Content_withoutStopWords_nltk:
    total_words_list_after_cleaning_nltk.append(len(word_list))

total_words_list_after_cleaning_nltk = np.array(total_words_list_after_cleaning_nltk)
total_character_count_after_cleaning = np.array(total_character_count_after_cleaning)

# print(total_words_list_after_cleaning_nltk)


# Average Sentence Length

total_no_of_sentences = []
for lst in tokenized_content:
      total_no_of_sentences.append(lst.count('.')+lst.count('?')+lst.count('!'))

total_no_of_sentences = np.array(total_no_of_sentences)

average_senetence_length = np.divide(total_words_list_after_cleaning_nltk,total_no_of_sentences)

# Average Word Length

average_word_length = np.divide(total_character_count_after_cleaning,total_words_list_after_cleaning_nltk)

# print(total_no_of_sentences)
print(average_word_length)

# Percentage of Complex words
no_of_complex_words = []

# coount syllable
syllable_count = []
def syllables(word):
    count = 0
    vowels = 'aeiou'
    exception_end_Tup = ('a','e','i','o','u','ed','es','le','les')

    if word[0] in vowels:
        count +=1
    for index in range(1,len(word)):
        if word[index] in vowels and word[index-1] not in vowels:
            count +=1
    if word.endswith(exception_end_Tup):
        count -= 1
    if count == 0:
        count += 1

    return count

for words in Content_withoutStopWords_nltk:
    i=0
    s_count = 0
    for word in words:
       syll_count = syllables(word)
       s_count += syll_count
       if syll_count >2:
           i+=1
    no_of_complex_words.append(i)
    syllable_count.append(s_count)
# print(Content_withoutStopWords_nltk)
# print(no_of_complex_words)
syllable_count = np.array(syllable_count)
print(syllable_count)
no_of_complex_words = np.array(no_of_complex_words)
percentage_complex_words = np.divide(no_of_complex_words,total_words_list_after_cleaning_nltk)*100

#syllable count per word

syllable_count_per_word = np.divide(syllable_count,total_words_list_after_cleaning_nltk)

# print(percentage_complex_words)

# Fog Index

fog_index = 0.4*(np.add(percentage_complex_words,average_senetence_length))

# print(fog_index)

# Personal Pronouns

# print(personal_pronoun_count)



[7.40100251 6.726      7.39700375 7.31046614 7.45692884 7.49122807
 6.72099853 6.88302752 7.20332481 6.92060491 6.71767497 6.72630458
 6.20528211 6.55366269 6.66320755 6.84010152 7.08145766 6.14109347
 6.63995726 6.12190083 6.38148984 6.84776903 6.86272727 6.96124031
 6.97201767 6.93039773 7.04143337 6.83733333 6.73969849 6.46312684
 6.67594937 6.85826772 6.53225806 7.07878788 6.59067358 6.39962121
 7.14359862 6.36835279 7.17412935 6.31419285 6.63627354 6.62078652
 6.72019465 6.52       6.16626506 6.49551752 6.33817595 7.30681818
 6.24062096 6.65681818 6.55475207 7.25901639 6.32876712 6.8381295
 7.46171694 6.97368421 6.70588235 6.75672215 6.62376238 6.57677165
 6.79283887 6.19313305 6.18303571 6.95431472 7.16315789 7.3655914
 6.15023474 6.39449541 7.46676096 6.67880795 6.60568384 7.18227092
 6.38375973 6.29221557 6.44827586 6.31532847 6.30991736 6.58811262
 6.3243762  5.98109966 6.16256831 6.78091873 6.3700361  6.48081535
 6.95794393 7.10699001 6.68944099 5.95991189 6.7447479  7.111228

## Saving all the calculated terms to the CSV file

In [16]:
output = pd.read_csv('/content/drive/MyDrive/Data/Output.csv')
# print(output['URL_ID'])
# print(output.columns)

output['POSITIVE SCORE'] = PositiveScore
output['NEGATIVE SCORE'] = NegativeScore
output['POLARITY SCORE'] = Polarity_score
output['SUBJECTIVITY SCORE'] = Subjectivity_score
output['AVG SENTENCE LENGTH'] = average_senetence_length
output['PERCENTAGE OF COMPLEX WORDS'] = percentage_complex_words
output['FOG INDEX'] = fog_index
output['AVG NUMBER OF WORDS PER SENTENCE'] = average_senetence_length ##???
output['COMPLEX WORD COUNT'] = no_of_complex_words
output['WORD COUNT'] = total_words_list_after_cleaning_nltk
output['SYLLABLE PER WORD'] = syllable_count_per_word
output['PERSONAL PRONOUNS'] = personal_pronoun_count
output['AVG WORD LENGTH'] = average_word_length

output.to_csv('/content/drive/MyDrive/Data/Output Data Structure.csv', index=False)
